In [1]:
import numpy as np
import cv2

funkcja do obliczania f1 w kolejnych przykladach

In [2]:
def calculate_metrics(TP, FP, FN):
    P = TP/(TP+FP) if (TP+FP) > 0 else 0
    R = TP/(TP+FN) if (TP+FN) > 0 else 0
    F1 = 2*P*R/(P+R) if (P+R) > 0 else 0
    return F1

3.1 mean i median - pedestrian

In [36]:
#wymiary ramki w sekwencji (na filmie sa takie same)
first_frame = cv2.imread('pedestrian/input/in000300.jpg', cv2.IMREAD_GRAYSCALE)
YY, XX = first_frame.shape
N = 60
buf = np.zeros((YY, XX, N), np.uint8 )
iN = 0
filled_frames = 0

#liczniki dla F1
TP_mean, TN_mean, FP_mean, FN_mean = 0, 0, 0, 0
TP_median, TN_median, FP_median, FN_median = 0, 0, 0, 0

step = 2
kernel = np.ones((3, 3))

for i in range(303, 1100, step):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)
    if GT is None:
        continue

    #aktualizacja bufora
    buf[:, :, iN] = IG
    iN = (iN + 1) % N  #inkrementacja wskaznika i zerowanie gdy jest rowny N

    # # Obliczanie średniej i mediany tła
    # mean_BG = np.mean(buf, axis=2).astype(np.uint8)
    # median_BG = np.median(buf, axis=2).astype(np.uint8)
    # Obliczenie średniej i mediany tła - tylko z faktycznie zapisanych ramek

    filled_frames = min(filled_frames + 1, N) 
    mean_BG = np.mean(buf[:, :, :filled_frames], axis=2).astype(np.uint8)
    median_BG = np.median(buf[:, :, :filled_frames], axis=2).astype(np.uint8)

    #odejmowanie tła
    diff_mean = cv2.absdiff(IG, mean_BG)
    diff_median = cv2.absdiff(IG, median_BG)

    #binaryzacja
    B_mean = (diff_mean > 15).astype(np.uint8) * 255
    B_median = (diff_median > 15).astype(np.uint8) * 255

    #operacje morfologiczne
    B_mean = cv2.medianBlur(B_mean, 7)
    B_mean = cv2.erode(B_mean, kernel, iterations=1)
    B_mean = cv2.dilate(B_mean, kernel, iterations=1)

    B_median = cv2.medianBlur(B_median, 7)
    B_median = cv2.erode(B_median, kernel, iterations=1)
    B_median = cv2.dilate(B_median, kernel, iterations=1)

    #konwersja groundtruth na wartości binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 255))
    TN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 0))
    FP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 0))
    FN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 255))

    TP_median += np.sum(np.logical_and(B_median == 255, GTB == 255))
    TN_median += np.sum(np.logical_and(B_median == 0, GTB == 0))
    FP_median += np.sum(np.logical_and(B_median == 255, GTB == 0))
    FN_median += np.sum(np.logical_and(B_median == 0, GTB == 255))

    cv2.imshow("mean background sub", B_mean)
    cv2.imshow("median background sub", B_median)
    cv2.imshow("groundtruth", GTB)

    if cv2.waitKey(10) == 27:  #27 to kod klawisza esc
        break
        
cv2.destroyAllWindows()


F1_mean = calculate_metrics(TP_mean, FP_mean, FN_mean)
F1_median = calculate_metrics(TP_median, FP_median, FN_median)

# Wypisanie wyników
print(f"F1-mean: {F1_mean:.4f}")
print(f"F1-median: {F1_median:.4f}")

F1-mean: 0.6878
F1-median: 0.8349


3.1 mean i median - office

In [17]:
#wymiary ramki w sekwencji (na filmie sa takie same)
first_frame = cv2.imread('office/input/in%06d.jpg' % 300, cv2.IMREAD_GRAYSCALE)
YY, XX = first_frame.shape
N = 60
buf = np.zeros((YY, XX, N), np.uint8 )
iN = 0
filled_frames = 0

#liczniki dla F1
TP_mean, TN_mean, FP_mean, FN_mean = 0, 0, 0, 0
TP_median, TN_median, FP_median, FN_median = 0, 0, 0, 0

step = 2
kernel = np.ones((3, 3))

for i in range(570, 1100, step):
    I = cv2.imread('office/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread('office/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)
    if GT is None:
        continue

    #aktualizacja bufora
    buf[:, :, iN] = IG
    iN = (iN + 1) % N  #inkrementacja wskaznika i zerowanie gdy jest rowny N

    filled_frames = min(filled_frames + 1, N) 
    mean_BG = np.mean(buf[:, :, :filled_frames], axis=2).astype(np.uint8)
    median_BG = np.median(buf[:, :, :filled_frames], axis=2).astype(np.uint8)

    #odejmowanie tła
    diff_mean = cv2.absdiff(IG, mean_BG)
    diff_median = cv2.absdiff(IG, median_BG)

    #binaryzacja
    B_mean = (diff_mean > 15).astype(np.uint8) * 255
    B_median = (diff_median > 15).astype(np.uint8) * 255

    #filtracja i morfologia
    B_mean = cv2.medianBlur(B_mean, 7)
    B_mean = cv2.erode(B_mean, kernel, iterations=1)
    B_mean = cv2.dilate(B_mean, kernel, iterations=2)

    B_median = cv2.medianBlur(B_median, 7)
    B_median = cv2.erode(B_median, kernel, iterations=1)
    B_median = cv2.dilate(B_median, kernel, iterations=2)

    #konwersja groundtruth na wartości binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 255))
    TN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 0))
    FP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 0))
    FN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 255))

    TP_median += np.sum(np.logical_and(B_median == 255, GTB == 255))
    TN_median += np.sum(np.logical_and(B_median == 0, GTB == 0))
    FP_median += np.sum(np.logical_and(B_median == 255, GTB == 0))
    FN_median += np.sum(np.logical_and(B_median == 0, GTB == 255))

    cv2.imshow("mean background sub", B_mean)
    cv2.imshow("median background sub", B_median)
    cv2.imshow("groundtruth", GTB)

    if cv2.waitKey(10) == 27:  #27 to kod klawisza esc
        break
        
cv2.destroyAllWindows()


F1_mean = calculate_metrics(TP_mean, FP_mean, FN_mean)
F1_median = calculate_metrics(TP_median, FP_median, FN_median)

# Wypisanie wyników
print(f"F1-mean: {F1_mean:.4f}")
print(f"F1-median: {F1_median:.4f}")

F1-mean: 0.6253
F1-median: 0.6518


3.2 aproksymacja mean i median

In [37]:
image = cv2.imread("pedestrian/input/in000300.jpg")
YY, XX, _ = image.shape  #wysokość, szerokość, liczba kanałów
N = 60
alpha = 0.01  #parametr wagowy (zwykle 0.01-0.05)

BGN_mean = np.float64(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))  #pierwsza ramka to model tla dla sredniej
BGN_median = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  #pierwsza ramka jako model tla dla mediany

TP_mean, TN_mean, FP_mean, FN_mean = 0, 0, 0, 0
TP_median, TN_median, FP_median, FN_median = 0, 0, 0, 0

kernel = np.ones((3, 3))
step = 2

for i in range(303, 1100, step):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)
    if GT is None:
        continue

    #aproksymacja średniej
    BGN_mean = alpha * IG + (1 - alpha) * BGN_mean
    diff_mean = np.abs(IG.astype('int') - BGN_mean.astype('int')) #odejmowanie tla od biezacej sceny
    B_mean = (diff_mean > 10).astype('uint8') * 255

    #aproksymacja mediany
    BGN_median = BGN_median + np.where(BGN_median < IG, 1, np.where(BGN_median > IG, -1, 0))
    diff_median = np.abs(IG.astype('int') - BGN_median.astype('int')) #odejmowanie tla od biezacej sceny
    B_median = (diff_median > 10).astype('uint8') * 255

    #filtracja i morfologia
    B_mean = cv2.medianBlur(B_mean, 5)
    B_mean = cv2.erode(B_mean, kernel, iterations=1)
    B_mean = cv2.dilate(B_mean, kernel, iterations=1)

    B_median = cv2.medianBlur(B_median, 5)
    B_median = cv2.erode(B_median, kernel, iterations=1)
    B_median = cv2.dilate(B_median, kernel, iterations=1)

    #konwersja groundtruth na wartości binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 255))
    TN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 0))
    FP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 0))
    FN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 255))

    TP_median += np.sum(np.logical_and(B_median == 255, GTB == 255))
    TN_median += np.sum(np.logical_and(B_median == 0, GTB == 0))
    FP_median += np.sum(np.logical_and(B_median == 255, GTB == 0))
    FN_median += np.sum(np.logical_and(B_median == 0, GTB == 255))

    cv2.imshow("mean background sub", B_mean)
    cv2.imshow("median background sub", B_median)
    cv2.imshow("groundtruth", GTB)

    if cv2.waitKey(10) == 27:  # 27 to kod klawisza ESC
        break


cv2.destroyAllWindows()


F1_mean = calculate_metrics(TP_mean, FP_mean, FN_mean)
F1_median = calculate_metrics(TP_median, FP_median, FN_median)

print(f"F1-mean: {F1_mean:.4f}")
print(f"F1-median: {F1_median:.4f}")


F1-mean: 0.8144
F1-median: 0.8608


3.3 aktualizacja z zapisem poprzednich masek

In [38]:
N = 60  
threshold = 15  #prog roznicy maski binarnej
kernel = np.ones((3, 3))
step = 2  

first_frame = cv2.imread('pedestrian/input/in000300.jpg', cv2.IMREAD_GRAYSCALE)
YY, XX = first_frame.shape
buf = np.zeros((YY, XX, N), np.uint8) 
iN = 0
filled_frames = 0

TP_mean, TN_mean, FP_mean, FN_mean = 0, 0, 0, 0
TP_median, TN_median, FP_median, FN_median = 0, 0, 0, 0

#maski obiektów z poprzednich klatek
prev_mask_mean = prev_mask_median = None

for i in range(303, 1100, step):
    I = cv2.imread(f'pedestrian/input/in{i:06d}.jpg')
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #maska referencyjna
    GT = cv2.imread(f'pedestrian/groundtruth/gt{i:06d}.png', cv2.IMREAD_GRAYSCALE)
    if GT is None:
        continue

    buf[:, :, iN] = IG
    iN = (iN + 1) % N 

    filled_frames = min(filled_frames + 1, N) 
    mean_BG = np.mean(buf[:, :, :filled_frames], axis=2).astype(np.uint8)  
    median_BG = np.median(buf[:, :, :filled_frames], axis=2).astype(np.uint8) 

    diff_mean = cv2.absdiff(IG, mean_BG)
    diff_median = cv2.absdiff(IG, median_BG)

    #binarna maska tła
    B_mean = (diff_mean > threshold).astype(np.uint8) * 255
    B_median = (diff_median > threshold).astype(np.uint8) * 255

    #sprawdzamy poprzednią maskę
    if prev_mask_mean is not None:
        B_mean = np.bitwise_and(B_mean, np.bitwise_not(prev_mask_mean))  #nie aktualizujemy tła gdzie były obiekty w poprzedniej klatce
    if prev_mask_median is not None:
        B_median = np.bitwise_and(B_median, np.bitwise_not(prev_mask_median))

    B_mean = cv2.medianBlur(B_mean, 7)
    B_mean = cv2.erode(B_mean, kernel, iterations=1)
    B_mean = cv2.dilate(B_mean, kernel, iterations=1)

    B_median = cv2.medianBlur(B_median, 7)
    B_median = cv2.erode(B_median, kernel, iterations=1)
    B_median = cv2.dilate(B_median, kernel, iterations=1)

    #konwersja groundtruth na wartości binarne
    GTB = (GT > 128).astype('uint8') * 255

    TP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 255))
    TN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 0))
    FP_mean += np.sum(np.logical_and(B_mean == 255, GTB == 0))
    FN_mean += np.sum(np.logical_and(B_mean == 0, GTB == 255))

    TP_median += np.sum(np.logical_and(B_median == 255, GTB == 255))
    TN_median += np.sum(np.logical_and(B_median == 0, GTB == 0))
    FP_median += np.sum(np.logical_and(B_median == 255, GTB == 0))
    FN_median += np.sum(np.logical_and(B_median == 0, GTB == 255))

    #zapamiętywanie poprzednich masek
    prev_mask_mean = B_mean
    prev_mask_median = B_median

    # cv2.imshow("mean background sub", B_mean)
    # cv2.imshow("median background sub", B_median)
    # cv2.imshow("groundtruth", GTB)

    # if cv2.waitKey(10) == 27:  # 27 to kod klawisza ESC
    #     break

# cv2.destroyAllWindows()

F1_mean = calculate_metrics(TP_mean, FP_mean, FN_mean)
F1_median = calculate_metrics(TP_median, FP_median, FN_median)

# Wypisanie wyników
print(f"F1-mean: {F1_mean:.4f}")
print(f"F1-median: {F1_median:.4f}")


F1-mean: 0.4887
F1-median: 0.5556


3.4 GMM/MOG

In [39]:
step = 2

#tworzenie obiektu BackgroundSubtractorMOG2
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)

TP, FP, FN = 0, 0, 0

for i in range(303, 1100, step):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #zastosowanie obiektu BackgroundSubtractor do aktualnej ramki
    fgmask = fgbg.apply(I)

    #maska referencyjna
    GT = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)
    if GT is None:
        continue
    GT = (GT > 128).astype(np.uint8) * 255  # Konwersja do binarnej maski

    TP += np.sum((fgmask == 255) & (GT == 255))  
    FP += np.sum((fgmask == 255) & (GT == 0))    
    FN += np.sum((fgmask == 0) & (GT == 255))    

    cv2.imshow('foreground mask', fgmask)
    cv2.imshow('original video', I)

    if cv2.waitKey(30) & 0xFF == 27:
        break


cv2.destroyAllWindows()

F1 = calculate_metrics(TP, FP, FN)
print(f"F1: {F1:.4f}")


F1: 0.7559


3.5 KNN

In [40]:
step = 2

#tworzenie obiektu BackgroundSubtractorKNN
fgbg = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=500, detectShadows=False)

TP, FP, FN = 0, 0, 0

for i in range(303, 1100, step):
    I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
    IG = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

    #zastosowanie obiektu BackgroundSubtractor do aktualnej ramki
    fgmask = fgbg.apply(I)

    GT = cv2.imread('pedestrian/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)
    if GT is None:
        continue
    GT = (GT > 128).astype(np.uint8) * 255 

    TP += np.sum((fgmask == 255) & (GT == 255))  
    FP += np.sum((fgmask == 255) & (GT == 0))    
    FN += np.sum((fgmask == 0) & (GT == 255))    

    cv2.imshow('foreground mask', fgmask)
    cv2.imshow('original video', I)

    if cv2.waitKey(30) & 0xFF == 27:  
        break

F1 = calculate_metrics(TP, FP, FN)
print(f"F1: {F1:.4f}")

cv2.destroyAllWindows()


F1: 0.5323
